## Libraries

In [1]:
#This code is for adaptive GPU usage
import keras.backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

Using TensorFlow backend.


In [4]:
#!pip3 install cntk-gpu

In [6]:
#!sudo pip3 install -U scikit-learn

[sudo] password for divi: 


In [6]:
#drive.mount('/content/gdrive/')

In [7]:
#!ls 'gdrive/My Drive/Microsoft AI'

## Loading Data

In [6]:
# archive = zipfile.ZipFile('gdrive/My Drive/Microsoft AI/data.zip', 'r')
# data = pd.read_csv(archive.open("data.tsv"),delimiter='\t',header=None)
# data.columns = ['queryID','query','response','label','labelID']
# data.isnull().values.any()

False

In [0]:
# sample = data.iloc[:10000].copy(deep = True)

In [8]:
# split = int(10*((sample.shape[0]*0.7)//10))
# print(split)

7000


In [0]:
# train = sample[:split]
# validation = sample[split:]

In [0]:
# train.to_csv("gdrive/My Drive/Microsoft AI/DL approach/traindata.tsv",sep='\t', index=False,header=None)
# validation.to_csv("gdrive/My Drive/Microsoft AI/DL approach/validationdata.tsv",sep='\t', index=False,header=None)

## Text2ctf

In [2]:
import re

#Initialize Global variables 
GloveEmbeddings = {}
max_query_words = 12
max_passage_words = 50
emb_dim = 50
#The following method takes Glove Embedding file and stores all words and their embeddings in a dictionary
def loadEmbeddings(embeddingfile):
    global GloveEmbeddings,emb_dim

    fe = open(embeddingfile,"r",encoding="utf-8",errors="ignore")
    for line in fe:
        tokens= line.strip().split()
        word = tokens[0]
        vec = tokens[1:]
        vec = " ".join(vec)
        GloveEmbeddings[word]=vec
    #Add Zerovec, this will be useful to pad zeros, it is better to experiment with padding any non-zero constant values also.
    GloveEmbeddings["zerovec"] = "0.0 "*emb_dim
    fe.close()


def TextDataToCTF(inputfile,outputfile,isEvaluation):
    global GloveEmbeddings,emb_dim,max_query_words,max_passage_words

    f = open(inputfile,"r",encoding="utf-8",errors="ignore")  # Format of the file : query_id \t query \t passage \t label \t passage_id
    fw = open(outputfile,"w",encoding="utf-8")
    for line in f:
        tokens = line.strip().lower().split("\t")
        query_id,query,passage,label = tokens[0],tokens[1],tokens[2],tokens[3]

        #****Query Processing****
        words = re.split('\W+', query)
        words = [x for x in words if x] # to remove empty words 
        word_count = len(words)
        remaining = max_query_words - word_count  
        if(remaining>0):
            words += ["zerovec"]*remaining # Pad zero vecs if the word count is less than max_query_words
        words = words[:max_query_words] # trim extra words
        #create Query Feature vector 
        query_feature_vector = ""
        for word in words:
            if(word in GloveEmbeddings):
                query_feature_vector += GloveEmbeddings[word]+" "
            else:
                query_feature_vector += GloveEmbeddings["zerovec"]+" "  #Add zerovec for OOV terms
        query_feature_vector = query_feature_vector.strip() 

        #***** Passage Processing **********
        words = re.split('\W+', passage)
        words = [x for x in words if x] # to remove empty words 
        word_count = len(words)
        remaining = max_passage_words - word_count  
        if(remaining>0):
            words += ["zerovec"]*remaining # Pad zero vecs if the word count is less than max_passage_words
        words = words[:max_passage_words] # trim extra words
        #create Passage Feature vector 
        passage_feature_vector = ""
        for word in words:
            if(word in GloveEmbeddings):
                passage_feature_vector += GloveEmbeddings[word]+" "
            else:
                passage_feature_vector += GloveEmbeddings["zerovec"]+" "  #Add zerovec for OOV terms
        passage_feature_vector = passage_feature_vector.strip() 

        #convert label
        label_str = " 1 0 " if label=="0" else " 0 1 " 

        if(not isEvaluation):
            fw.write("|qfeatures "+query_feature_vector+" |pfeatures "+passage_feature_vector+" |labels "+label_str+"\n")
        else:
            fw.write("|qfeatures "+query_feature_vector+" |pfeatures "+passage_feature_vector+"|qid "+str(query_id)+"\n")



if __name__ == "__main__":

    trainFileName = "traindata.tsv"
    validationFileName = "validationdata.tsv"
    EvaluationFileName = "processed_test.tsv"

    embeddingFileName = "glove.6B.50d.txt"

    loadEmbeddings(embeddingFileName)    

    # Convert Query,Passage Text Data to CNTK Text Format(CTF) using 50-Dimension Glove word embeddings 
    TextDataToCTF(trainFileName,"TrainData.ctf",False)
    print("Train Data conversion is done")
    TextDataToCTF(validationFileName,"ValidationData.ctf",False)
    print("Validation Data conversion is done")
    TextDataToCTF(EvaluationFileName,"EvaluationData.ctf",True)
    print("Evaluation Data conversion is done")






Train Data conversion is done
Validation Data conversion is done
Evaluation Data conversion is done


## CNN model

In [1]:
!ls

DL.ipynb


In [3]:
#!apt-get install openmpi-bin

E: Could not open lock file /var/lib/dpkg/lock - open (13: Permission denied)
E: Unable to lock the administration directory (/var/lib/dpkg/), are you root?


In [19]:
#!pip3 install cntk-gpu

    100% |████████████████████████████████| 478.8MB 27kB/s 


In [21]:
#!pip3 install https://cntk.ai/PythonWheel/CPU-Only/cntk-2.6-cp35-cp35m-linux_x86_64.whl

cntk-2.6-cp35-cp35m-linux_x86_64.whl is not a supported wheel on this platform.


In [3]:
from __future__ import print_function
import numpy as np
import sys
import os
import cntk as C
from cntk.io import MinibatchSource, CTFDeserializer, StreamDef, StreamDefs, INFINITELY_REPEAT, FULL_DATA_SWEEP
from sklearn.metrics import precision_recall_fscore_support
from sklearn.exceptions import UndefinedMetricWarning
import warnings
warnings.filterwarnings("ignore", category=UndefinedMetricWarning) 


/opt/anaconda/lib/python3.6/site-packages/cntk/cntk_py_init.py:90: UserWarning: 

################################################ Missing optional dependency (    MKL     ) ################################################
   CNTK may crash if the component that depends on those dependencies is loaded.
   Visit https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-Linux-Python#mkl for more information.
############################################################################################################################################

  warnings.warn(WARNING_MSG % ('    MKL     ', 'https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-Linux-Python#mkl'))
/opt/anaconda/lib/python3.6/site-packages/cntk/cntk_py_init.py:102: UserWarning: 

################################################ Missing optional dependency (   OpenCV   ) ################################################
   CNTK may crash if the component that depends on those dependencies is loaded.
   Visit https://do

In [4]:



#Initialize Global variables
validation_query_vectors = []
validation_passage_vectors = []
validation_labels = []   
q_max_words=12
p_max_words=50
emb_dim=50

## The following LoadValidationSet method reads ctf format validation file and creates query, passage feature vectors and also copies labels for each pair.
## the created vectors will be useful to find metrics on validation set after training each epoch which will be useful to decide the best model 
def LoadValidationSet(validationfile):
    f = open(validationfile,'r',encoding="utf-8")
    for line in f:
        tokens = line.strip().split("|")  
        #tokens[0] will be empty token since the line is starting with |
        x1 = tokens[1].replace("qfeatures","").strip() #Query Features
        x2 = tokens[2].replace("pfeatures","").strip() # Passage Features
        y = tokens[3].replace("labels","").strip() # labels
        x1 = [float(v) for v in x1.split()]
        x2 = [float(v) for v in x2.split()]
        y = [int(w) for w in y.split()]        
        y = y[1] # label will be at index 1, i.e. if y = "1 0" then label=0 else if y="0 1" then label=1

        validation_query_vectors.append(x1)
        validation_passage_vectors.append(x2)
        validation_labels.append(y)

        #print("1")
    
    print("Validation Vectors are created")

#The following method defines a CNN network which has series of convolution and max pooling steps on query features and passage features and then a merge step and it follows a fully connected layer
def cnn_network(queryfeatures, passagefeatures, num_classes):
    with C.layers.default_options(activation=C.ops.relu, pad=False):
        convA1 = C.layers.Convolution2D((3,10),4,pad=False,activation=C.tanh,name='convA1')(queryfeatures) #input : 12*50 #output : 4*10*41
        poolA1 = C.layers.MaxPooling((2,3),(2,3),name='poolA1')(convA1)  #output : 4*5*13
        convA2 = C.layers.Convolution2D((2,4),2,pad=False,activation=C.tanh,name='convA2')(poolA1) #output : 2*4*10
        poolA2 = C.layers.MaxPooling((2,2),(2,2),name='poolA2')(convA2)  #output : 2*2*5
        denseA = C.layers.Dense(num_classes*num_classes,activation=C.tanh,name='denseA')(poolA2)  # output : 4
         
        convB1 = C.layers.Convolution2D((5,10),4,pad=False,activation=C.tanh,name='convB1')(passagefeatures) #input : 50*50  #output : 4*46*41
        poolB1 = C.layers.MaxPooling((5,5),(5,5),name='poolB1')(convB1) #output : 4*9*8
        convB2 = C.layers.Convolution2D((3,3),2,pad=False,activation=C.tanh,name='convB2')(poolB1) #output : 2*7*6
        poolB2 = C.layers.MaxPooling((2,2),(2,2),name='poolB2')(convB2) #output : 2*3*3
        denseB = C.layers.Dense(num_classes*num_classes,activation=C.tanh,name='denseB')(poolB2)  # output : 4


        mergeQP     = C.element_times(denseA,denseB) # output : 4

        model   = C.layers.Dense(num_classes, activation=C.softmax,name="overall")(mergeQP) #outupt : 2 
        

    return model

def create_reader(path, is_training, query_total_dim, passage_total_dim, label_total_dim):
    return MinibatchSource(CTFDeserializer(path, StreamDefs( queryfeatures = StreamDef(field='qfeatures', shape=query_total_dim,is_sparse=False), 
                                                            passagefeatures = StreamDef(field='pfeatures', shape=passage_total_dim,is_sparse=False), 
                                                            labels   = StreamDef(field='labels', shape=label_total_dim,is_sparse=False)
                                                            )), 
                           randomize=is_training, max_sweeps = INFINITELY_REPEAT if is_training else FULL_DATA_SWEEP)

def TrainAndValidate(trainfile):

    #*****Hyper-Parameters******
    q_max_words= 12
    p_max_words = 50
    emb_dim = 50
    num_classes = 2
    minibatch_size = 250
    epoch_size = 10 #No.of samples in training set
    total_epochs = 60 #Total number of epochs to run
    query_total_dim = q_max_words*emb_dim
    label_total_dim = num_classes
    passage_total_dim = p_max_words*emb_dim


    #****** Create placeholders for reading Training Data  ***********
    query_input_var =  C.ops.input_variable((1,q_max_words,emb_dim),np.float32,is_sparse=False)
    passage_input_var =  C.ops.input_variable((1,p_max_words,emb_dim),np.float32,is_sparse=False)
    output_var = C.input_variable(num_classes,np.float32,is_sparse = False)
    train_reader = create_reader(trainfile, True, query_total_dim, passage_total_dim, label_total_dim)
    input_map = { query_input_var : train_reader.streams.queryfeatures, passage_input_var:train_reader.streams.passagefeatures, output_var : train_reader.streams.labels}

    # ********* Model configuration *******
    model_output = cnn_network(query_input_var, passage_input_var, num_classes)
    loss = C.binary_cross_entropy(model_output, output_var)
    pe = C.classification_error(model_output, output_var)
    lr_per_minibatch = C.learning_rate_schedule(0.03, C.UnitType.minibatch)   
    learner = C.adagrad(model_output.parameters, lr=lr_per_minibatch)
    progress_printer = C.logging.ProgressPrinter(tag='Training', num_epochs=total_epochs)

    #************Create Trainer with model_output object, learner and loss parameters*************  
    trainer = C.Trainer(model_output, (loss, pe), learner, progress_printer)
    C.logging.log_number_of_parameters(model_output) ; print()

    # **** Train the model in batchwise mode *****
    for epoch in range(total_epochs):       # loop over epochs
        print("Epoch : ",epoch)
        sample_count = 0
        while sample_count < epoch_size:  # loop over minibatches in the epoch
            data = train_reader.next_minibatch(min(minibatch_size, epoch_size - sample_count), input_map=input_map) # fetch minibatch.
            trainer.train_minibatch(data)        # training step
            sample_count += data[output_var].num_samples   # count samples processed so far

        trainer.summarize_training_progress()
                
        #model_output.save("CNN_{}.dnn".format(epoch)) # Save the model for every epoch
    
        #*** Find metrics on validation set after every epoch ******#  (Note : you can skip doing this for every epoch instead to optimize the time, do it after every k epochs)
        predicted_labels=[]
        for i in range(len(validation_query_vectors)):
            queryVec   = np.array(validation_query_vectors[i],dtype="float32").reshape(1,q_max_words,emb_dim)
            passageVec = np.array(validation_passage_vectors[i],dtype="float32").reshape(1,p_max_words,emb_dim)
            scores = model_output(queryVec,passageVec)[0]   # do forward-prop on model to get score  
            predictLabel = 1 if scores[1]>=scores[0] else 0
            predicted_labels.append(predictLabel) 
        metrics = precision_recall_fscore_support(np.array(validation_labels), np.array(predicted_labels), average='binary')
        #print("precision : "+str(metrics[0])+" recall : "+str(metrics[1])+" f1 : "+str(metrics[2])+"\n")
        model_output.save("CNN.dnn")


    return model_output

## The following GetPredictionOnEvalSet method reads all query passage pair vectors from CTF file and does forward prop with trained model to get similarity score
## after getting scores for all the pairs, the output will be written into submission file. 
def GetPredictionOnEvalSet(model,testfile,submissionfile):
    global q_max_words,p_max_words,emb_dim

    f = open(testfile,'r',encoding="utf-8")
    all_scores={} # Dictionary with key = query_id and value = array of scores for respective passages
    for line in f:
        tokens = line.strip().split("|")  
        #tokens[0] will be empty token since the line is starting with |
        x1 = tokens[1].replace("qfeatures","").strip() #Query Features
        x2 = tokens[2].replace("pfeatures","").strip() # Passage Features
        query_id = tokens[3].replace("qid","").strip() # Query_id
        x1 = [float(v) for v in x1.split()]
        x2 = [float(v) for v in x2.split()]    
        queryVec   = np.array(x1,dtype="float32").reshape(1,q_max_words,emb_dim)
        passageVec = np.array(x2,dtype="float32").reshape(1,p_max_words,emb_dim)
        score = model(queryVec,passageVec)[0][1] # do forward-prop on model to get score
        if(query_id in all_scores):
            all_scores[query_id].append(score)
        else:
            all_scores[query_id] = [score]
    fw = open(submissionfile,"w",encoding="utf-8")
    for query_id in all_scores:
        scores = all_scores[query_id]
        scores_str = [str(sc) for sc in scores] # convert all scores to string values
        scores_str = "\t".join(scores_str) # join all scores in list to make it one string with  tab delimiter.  
        fw.write(query_id+"\t"+scores_str+"\n")
    fw.close()

    
if __name__ == "__main__":

    trainSetFileName = "TrainData.ctf"
    validationSetFileName = "ValidationData.ctf"
    testSetFileName = "EvaluationData.ctf"
    submissionFileName = "dl_answer.tsv"
   
    LoadValidationSet(validationSetFileName)    #Load Validation Query, Passage Vectors from Validation CTF File
    model = TrainAndValidate(trainSetFileName) # Training and validation methods    
    GetPredictionOnEvalSet(model,testSetFileName,submissionFileName) # Get Predictions on Evaluation Set



    

Validation Vectors are created
Training 638 parameters in 14 parameter tensors.

Epoch :  0
Learning rate per minibatch: 0.03
Finished Epoch[1 of 60]: [Training] loss = 1.374198 * 10, metric = 20.00% * 10 7.654s (  1.3 samples/s);
Epoch :  1
Finished Epoch[2 of 60]: [Training] loss = 1.340756 * 10, metric = 10.00% * 10 18.688s (  0.5 samples/s);
Epoch :  2
Finished Epoch[3 of 60]: [Training] loss = 1.321612 * 10, metric = 20.00% * 10 18.284s (  0.5 samples/s);
Epoch :  3
Finished Epoch[4 of 60]: [Training] loss = 1.310757 * 10, metric = 20.00% * 10 18.302s (  0.5 samples/s);
Epoch :  4
Finished Epoch[5 of 60]: [Training] loss = 1.262159 * 10, metric = 10.00% * 10 18.270s (  0.5 samples/s);
Epoch :  5
Finished Epoch[6 of 60]: [Training] loss = 1.282151 * 10, metric = 20.00% * 10 18.318s (  0.5 samples/s);
Epoch :  6
Finished Epoch[7 of 60]: [Training] loss = 1.166369 * 10, metric = 0.00% * 10 18.307s (  0.5 samples/s);
Epoch :  7
Finished Epoch[8 of 60]: [Training] loss = 1.164882 * 10,